In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns; sns.set()
import datetime as dt
sns.set(rc={'figure.figsize':(20, 10)})

In [2]:
Indicator=pd.read_csv("/home/tirayr/pythonscripts/MyPjOne/data/work/Market_Indicator_data_nudget_AIRBUS.out"
                   ,"|"
                   ,engine='python',index_col=1
                   ,parse_dates=True)
News=pd.read_csv("/home/tirayr/pythonscripts/MyPjOne/data/daily/20200517/Final_FinNews_Analysed/AIRBUS_20170715_20200424.csv"
                   ,"|"
                   ,engine='python',index_col=0
                   ,parse_dates=True)

In [3]:
# MyDataIndicator.head(n=5)
News.head()

,sentence,logit,prediction,sentiment_score
time_index,,,,
2020-04-20 13:04:38,Exclusive: Airbus puts six jets made for AirAs...,[0.48549378 0.01719717 0.49730912],neutral,0.468297
2020-04-17 15:44:57,"Airbus furloughs 3,000 French aerospace worker...",[0.3995303 0.13287124 0.46759847],neutral,0.266659
2020-04-17 15:11:33,"BRIEF-Airbus confirms 3,000 French employees a...",[0.01594882 0.8628845 0.12116661],negative,-0.846936
2020-04-16 13:55:20,Airbus evaluates requests for delays to jet de...,[0.01915266 0.9468561 0.03399128],negative,-0.927703
2020-04-16 12:16:06,Airbus says has received many requests to defe...,[0.02301058 0.95120376 0.02578566],negative,-0.928193


In [4]:
News=News.groupby(['time_index']).agg(lambda x : x.mean() if x.dtype=='float64' else ' '.join(x))
News.loc[(News['sentiment_score']<0.5) & (News['sentiment_score'])>-0.5,'prediction'] ='neutral'
News.loc[News['sentiment_score']>0.5,'prediction'] ='positive'
News.loc[News['sentiment_score']>0.5,'prediction'] ='positive'

In [8]:
startT=dt.time(9,0,0)
endT  =dt.time(15,30,0)
News['time'] = pd.to_datetime(News.index)
News_Filtered=News.loc[   (News['time'].dt.time>startT) 
                        & (News['time'].dt.time<endT) 
                        & (News['prediction'] != 'neutral')][['sentence','logit','prediction','sentiment_score']]
# News_Filtered

In [11]:
# # News=News.groupby(['time_index']).agg(lambda x : x.mean() if x.dtype=='float64' else ' '.join(x))
# std_df=Indicator
# std_df.dtypes
# std_df.index = pd.to_datetime(std_df.index)
# std_df.index = std_df.index.to_period('M')
# std_df=std_df.groupby([std_df.index]).agg(lambda x : x.std() if x.dtype=='float64' else ' '.join(x)) #, 'mean', 'std']
# std_df['30MIN_SUMMED_NUDGET']

# # new_df.index.set_names(['Year', 'Month'], inplace = True)
# # new_df.reset_index(inplace = True)
# # new_df

# def zscore(s):
#     return (s - np.mean(s)) / np.std(s)

In [44]:
# zscore(Indicator['30MIN_SUMMED_NUDGET']).abs().gt(2)
# News_Filtered.head()
# News_Filtered['']
Indicator['date']=Indicator.index.date
Indicator['time']=Indicator.index.time
News_Filtered['date']=News_Filtered.index.date
News_Filtered['time']=News_Filtered.index.time
event_time_diff=0.5
df_merge_difkey = pd.merge(Indicator, News_Filtered, left_on=Indicator['date'], right_on=News_Filtered['date'])
df_merge_difkey.loc[df_merge_difkey['time_x']-df_merge_difkey['time_y']<=datetime.timedelta(hours=event_time_diff)][['30MIN_SUMMED_NUDGET','sentiment_score','date_y','date_x','time_x','time_y']]

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [ ]:
News_Filtered['sentiment_score'].plot(linewidth=2,marker='o',linestyle='-',figsize=(50, 15))

In [ ]:
MyDataIndicator.loc['2020-02','30MIN_SUMMED_NUDGET'].plot(linewidth=0.5,marker='.', linestyle='-')

In [ ]:
MyDataNews.loc['2020-02','sentiment_score'].plot(linewidth=2,marker='o',linestyle='-',figsize=(10, 5))

In [ ]:

MyData_weekly_mean = MyData['30MIN_SUMMED_NUDGET'].resample('W').mean()
MyData_7d = MyData['30MIN_SUMMED_NUDGET'].rolling(7, center=True).mean()
# Start and end of the date range to extract
start, end = '2019-01', '2019-06'
# Plot daily, weekly resampled, and 7-day rolling mean time series together
fig, ax = plt.subplots()
ax.plot(MyData.loc[start:end, '30MIN_SUMMED_NUDGET'],
marker='.', linestyle='-', linewidth=0.5, label='Original')
ax.plot(MyData_weekly_mean.loc[start:end, '30MIN_SUMMED_NUDGET'],
marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')
ax.plot(MyData_7d.loc[start:end, '30MIN_SUMMED_NUDGET'],
marker='.', linestyle='-', label='7-d Rolling Mean')
ax.set_ylabel('Solar Production (GWh)')
ax.legend();

In [ ]:
ax=MyData['30MIN_SUMMED_NUDGET'].plot(linewidth=0.1, marker='.', alpha=0.5, linestyle='None', figsize=(20, 10))
ax.set_ylabel('TIME')

In [ ]:
cols_plot = ['30MIN_SUMMED_NUDGET']
axes = opsd_daily[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(11, 9), subplots=True)
for ax in axes:
    ax.set_ylabel('Daily Totals (GWh)')